In [ ]:
!pip install -q playwright nest_asyncio
!playwright install --with-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 16.0 MB/s eta 0:00:00
Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-se

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Patch the running event loop

async def fetch_page_content(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")
        content = await page.content()
        await browser.close()
        return content

# Run the coroutine safely in Colab
loop = asyncio.get_event_loop()
html_content = loop.run_until_complete(fetch_page_content("https://www.bookmywed.in/venue/delhi+wedding-resorts"))

print(html_content)


<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><meta content="BookMyWed" name="author">
    <!-- Libs -->
    
    <link hreflang="en" rel="stylesheet" href="/assets/libs/bootstrap-icons/font/bootstrap-icons.min.css"><link hreflang="en" rel="stylesheet" href="/assets/fonts/wedding-icon-font/flaticon.min.css"><link hreflang="en" rel="stylesheet" href="/assets/fonts/fontello/css/fontello.min.css"><link hreflang="en" rel="shortcut icon" href="/assets/images/logo.png">

    <!-- Theme CSS -->
    <link hreflang="en" rel="stylesheet" href="/assets/css/theme.min.css"><link hreflang="en" rel="stylesheet" href="/assets/css/custom.css"><link hreflang="en" rel="stylesheet" href="/assets/css/helper.min.css"><link hreflang="en" href="/assets/libs/flatpickr/flatpickr.min.css" rel="stylesheet"><title>
	Book My Wed - Wedding Resorts In Delhi
</title><link rel="preconnect" hreflang="en" href="https://fonts.googleapis.com

In [ ]:
from bs4 import BeautifulSoup
from tabulate import tabulate
import csv

soup = BeautifulSoup(html_content, "html.parser")

hotel_blocks = soup.find_all("div", class_="venue-list-card-content")

hotel_info = []
for block in hotel_blocks[:10]:
    # Hotel name
    name_div = block.find("div", attrs={"venue-name"})
    hotel_name = name_div.find("h3").get_text(strip=True) if name_div and name_div.find("h3") else "Name not available"

    # Location
    location_div = block.find("div", class_="location")
    location = location_div.get_text(strip=True) if location_div else "Location not available"

    # Venue type and capacity
    detail_divs = block.find_all("div", class_="location new-font1")
    venue_type = detail_divs[0].get_text(strip=True) if len(detail_divs) > 0 else "Venue type not available"
    capacity = detail_divs[1].get_text(strip=True) if len(detail_divs) > 1 else "Capacity not available"

    # Price per plate
    price = "Price not available"
    meta_details = block.find("div", class_="venue-meta-details")
    if meta_details:
        meta_list = meta_details.find("ul", class_="meta-list")
        if meta_list:
            first_li = meta_list.find("li")
            if first_li:
                price_span = first_li.find("span")
                if price_span:
                    raw_price = price_span.get_text(strip=True)
                    price = raw_price.replace("Per Plate", " Per Plate")

    hotel_info.append([hotel_name, location, venue_type, capacity, price])

# Save to CSV file
csv_filename = "hotel_data.csv"
with open(csv_filename, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Hotel Name", "Location", "Venue Type", "Capacity", "Price per Plate"])  # header
    writer.writerows(hotel_info)

print(f"\n✅ Data successfully written to '{csv_filename}'.")

# Optional: print nicely in tabular format for verification
print(tabulate(hotel_info, headers=["Hotel Name", "Location", "Venue Type", "Capacity", "Price per Plate"], tablefmt="grid"))



✅ Data successfully written to 'hotel_data.csv'.
+-----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+----------------+-------------------+
| Hotel Name                  | Location                                                                                                                                     | Venue Type                                                                               | Capacity       | Price per Plate   |
+=============================+==============================================================================================================================================+==========================================================================================+================+===================+
| Ashok Country Resort        | 110037 30

In [ ]:
from bs4 import BeautifulSoup

def extract_divs_with_images(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    result_divs = []

    # Find all divs that contain an <img> tag
    for div in soup.find_all('div'):
        if div.find('img'):  # Check if there's an <img> inside this <div>
            result_divs.append(str(div))  # Store the full div as a string

    return result_divs

# Example usage
divs_with_images = extract_divs_with_images(html)

# Print first 3 divs (just for preview)
for i, div in enumerate(divs_with_images[:3], 1):
    print(f"\n--- Venue {i} ---\n{div}")



--- Venue 1 ---
<div>
<div class="bg-white top-bar py-1">
<div class="container">
<div class="row">
<div class="col-md-12">
<div class="d-flex new-flex-top justify-content-between fs-5 align-items-center">
<div class="">
<div class="d-flex align-items-center">
<p class="mb-0 d-none d-md-block" style="text-wrap-mode: nowrap;">Welcome to book my wed.</p>
<a class="ms-2 new-border form-select form-select-sm cityMODAL" data-bs-target="#city1" data-bs-toggle="modal" href="javscript:void(0);">Delhi</a>
</div>
</div>
<div class="rightbar">
<p class="mb-0">
<a class="" href="https://play.google.com/store/apps/details?id=com.nextwebi.bookmywed&amp;pcampaignid=web_share" target="_blank"><i class="fa fa-mobile-alt"></i>Download Vendor App </a>
<span>|</span>
<a class="" href="/wishlist.aspx"><i class="fa-regular fa-heart"></i>Your Wishlist</a>
</p>
</div>
</div>
</div>
</div>
</div>
</div>
<div class="navbar-transparent bg-back border-white border-opacity-25" style="background: hsl(351.37deg 65.

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

nest_asyncio.apply()

# 🔹 Asynchronously fetch page content
async def fetch_page_content(city):
    city_url = city.lower().replace(" ", "+")  # e.g., "new delhi" → "new+delhi"
    url = f"https://www.bookmywed.in/venue/{city_url}+wedding-resorts"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url, wait_until="load", timeout=60000)
            content = await page.content()
        except Exception as e:
            print(f"Error loading page: {e}")
            content = ""
        await browser.close()
        return content

# 🔹 Extract venue <div> blocks that contain <img> tags
def extract_venue_divs_with_images(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    venue_data = []

    for div in soup.find_all('div'):
        img_tag = div.find('img')
        if img_tag and img_tag.get('src'):
            venue_info = {
                'image_url': img_tag['src'],
                'full_div_html': str(div)
            }
            venue_data.append(venue_info)

    return venue_data

# 🔹 Run full process for any city
def scrape_venue_images(city):
    loop = asyncio.get_event_loop()
    html = loop.run_until_complete(fetch_page_content(city))
    venue_list = extract_venue_divs_with_images(html)

    print(f"\nFound {len(venue_list)} venues with images in '{city.title()}':\n")
    for i, venue in enumerate(venue_list[:10], 1):  # Show first 10 for demo
        print(f"{i}. Image URL: {venue['image_url']}")
        # Uncomment below to print the full HTML of the div
        # print(venue['full_div_html'])

# 🔹 USER INPUT HERE
city_input = input("Enter city name: ")
scrape_venue_images(city_input)


Enter city name: Lucknow

Found 51 venues with images in 'Lucknow':

1. Image URL: /assets/images/logo-wedding.png
2. Image URL: /assets/images/logo-wedding.png
3. Image URL: /assets/images/logo-wedding.png
4. Image URL: https://imgcdn.bookmywed.in/UploadImages/00761619-4ff7-4c39-9035-8e704b894bfd_location.jpg
5. Image URL: https://imgcdn.bookmywed.in/UploadImages/00761619-4ff7-4c39-9035-8e704b894bfd_location.jpg
6. Image URL: https://imgcdn.bookmywed.in/UploadImages/9c204864-c587-4dd2-a3c8-e7a23374f7b4_location.jpg
7. Image URL: https://imgcdn.bookmywed.in/venuegallery/b149edad-da2b-44f3-9a4e-3a06976593062023-08-26 (1).jpg
8. Image URL: https://imgcdn.bookmywed.in/venuegallery/b149edad-da2b-44f3-9a4e-3a06976593062023-08-26 (1).jpg
9. Image URL: https://imgcdn.bookmywed.in/venuegallery/b149edad-da2b-44f3-9a4e-3a06976593062023-08-26 (1).jpg
10. Image URL: https://imgcdn.bookmywed.in/venuegallery/b149edad-da2b-44f3-9a4e-3a06976593062023-08-26 (1).jpg


In [ ]:
html_content = loop.run_until_complete(fetch_page_content("https://www.bookmywed.in/venue/delhi+wedding-resorts"))


In [ ]:
from google.colab import files
files.download("hotel_data_with_images.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q playwright nest_asyncio
!playwright install --with-deps

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import csv


Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,742 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,021 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security

In [ ]:
nest_asyncio.apply()

async def fetch_page_content(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")
        content = await page.content()
        await browser.close()
        return content

loop = asyncio.get_event_loop()
url = "https://www.bookmywed.in/venue/delhi+wedding-resorts"
html_content = loop.run_until_complete(fetch_page_content(url))


In [ ]:
soup = BeautifulSoup(html_content, "html.parser")
hotel_blocks = soup.find_all("div", class_="venue-list-card-content")

hotel_info = []
for block in hotel_blocks[:10]:
    name_div = block.find("div", attrs={"venue-name"})
    hotel_name = name_div.find("h3").get_text(strip=True) if name_div and name_div.find("h3") else "Name not available"

    location_div = block.find("div", class_="location")
    location = location_div.get_text(strip=True) if location_div else "Location not available"

    detail_divs = block.find_all("div", class_="location new-font1")
    venue_type = detail_divs[0].get_text(strip=True) if len(detail_divs) > 0 else "Venue type not available"
    capacity = detail_divs[1].get_text(strip=True) if len(detail_divs) > 1 else "Capacity not available"

    price = "Price not available"
    meta_details = block.find("div", class_="venue-meta-details")
    if meta_details:
        meta_list = meta_details.find("ul", class_="meta-list")
        if meta_list:
            first_li = meta_list.find("li")
            if first_li:
                price_span = first_li.find("span")
                if price_span:
                    raw_price = price_span.get_text(strip=True)
                    price = raw_price.replace("Per Plate", " Per Plate")

    hotel_info.append([hotel_name, location, venue_type, capacity, price])

# Save CSV
csv_filename = "hotel_data.csv"
with open(csv_filename, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Hotel Name", "Location", "Venue Type", "Capacity", "Price per Plate"])
    writer.writerows(hotel_info)


In [ ]:
from google.colab import files
files.download("hotel_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
nest_asyncio.apply()

async def fetch_page_content(city):
    city_url = city.lower().replace(" ", "+")
    url = f"https://www.bookmywed.in/venue/{city_url}+wedding-resorts"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")
        content = await page.content()
        await browser.close()
        return content

def scrape_venues_with_images(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    venue_cards = soup.find_all("div", class_="venue-list-card")

    venue_data = []
    for card in venue_cards[:10]:
        content = card.find("div", class_="venue-list-card-content")
        image_tag = card.find("img")

        name_div = content.find("div", attrs={"venue-name"}) if content else None
        hotel_name = name_div.find("h3").get_text(strip=True) if name_div and name_div.find("h3") else "Name not available"

        location_div = content.find("div", class_="location") if content else None
        location = location_div.get_text(strip=True) if location_div else "Location not available"

        detail_divs = content.find_all("div", class_="location new-font1") if content else []
        venue_type = detail_divs[0].get_text(strip=True) if len(detail_divs) > 0 else "Venue type not available"
        capacity = detail_divs[1].get_text(strip=True) if len(detail_divs) > 1 else "Capacity not available"

        price = "Price not available"
        meta_details = content.find("div", class_="venue-meta-details") if content else None
        if meta_details:
            meta_list = meta_details.find("ul", class_="meta-list")
            if meta_list:
                first_li = meta_list.find("li")
                if first_li:
                    price_span = first_li.find("span")
                    if price_span:
                        raw_price = price_span.get_text(strip=True)
                        price = raw_price.replace("Per Plate", " Per Plate")

        image_url = image_tag['src'] if image_tag and image_tag.has_attr('src') else "Image not available"

        venue_data.append([hotel_name, location, venue_type, capacity, price, image_url])
    return venue_data

# USER INPUT
city = "Delhi"  # Change this to any city like "Mumbai", "Jaipur", etc.
html_content = asyncio.get_event_loop().run_until_complete(fetch_page_content(city))
venue_data = scrape_venues_with_images(html_content)

# WRITE TO CSV
csv_filename = "hotel_data_with_images.csv"
with open(csv_filename, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Hotel Name", "Location", "Venue Type", "Capacity", "Price per Plate", "Image URL"])
    writer.writerows(venue_data)

print(f"\n✅ Data saved to {csv_filename}")

# DOWNLOAD IN COLAB
from google.colab import files
files.download(csv_filename)


✅ Data saved to hotel_data_with_images.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>